In [ ]:
#Data augmentation

#At every new epoch new random transformations will be applied
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

target_size = (300, 300)
batch_size = 20

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

#In validation set, we only rescale the data
validation_datagen = ImageDataGenerator(rescale=1./255)

# This is a generator that will read pictures found in 'train' and generate batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'pictures/train',  #This is the target directory
        target_size=(300, 300),  #All images will be resized to 300x300
        batch_size=batch_size, color_mode = "rgb", class_mode='categorical')

# This is a similar generator for validation data
validation_generator = validation_datagen.flow_from_directory(
        'pictures/validation',
        target_size=(300, 300), batch_size=batch_size, color_mode = "rgb", class_mode='categorical', shuffle = False)


In [ ]:
#******************** train own CNN model**********************

from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

train_num = 2585
val_num = 699

#Method 1 - build a new simple model and train

def create_base():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3))) #input layer
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

#The model so far outputs 3D feature maps (height, width, features)

    model.add(Flatten())  #Convert 3D feature maps to 1D feature vectors
    model.add(Dense(64)) #Usually be 64, 128, 256...
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6))
    model.add(Activation('softmax'))
    
# Compile
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['categorical_accuracy'])
    return model
    

In [ ]:
checkpoint_1 = ModelCheckpoint('own_model_best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')

model_1 = create_base()

model_1.fit_generator(
        train_generator,
        steps_per_epoch= train_num // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps= val_num // batch_size, callbacks = [checkpoint_1])

#model_1.save_weights('model_1_epochs.h5')  



In [ ]:
#******************** Transfer learning on pre-trained model**********************

#Method 2 - train on Xception + all layers

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

# Dimensions from the image after being resized
shape = [300, 300, 3]

model_xception = tf.keras.applications.xception.Xception(
                  include_top=False,
                  weights= 'imagenet',
                  input_shape=shape,
                  pooling='max'
                )
model_2 = tf.keras.Sequential()
model_2.add(model_xception)
model_2.add(tf.keras.layers.Flatten())
model_2.add(tf.keras.layers.Dense(6, activation='softmax'))

#Compile
model_2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['categorical_accuracy'])

model_2.summary()


In [ ]:
checkpoint_2 = ModelCheckpoint('Xception_all_layers.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')

model_2.fit_generator(
        train_generator,
        steps_per_epoch= train_num // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps= val_num // batch_size, callbacks = [checkpoint_1])

In [ ]:
#Method 3 - train on VGG + only on last four layers + a full connected layer

model_vgg = VGG16(include_top=False, weights='imagenet',input_shape=(300, 300, 3))
print('Model loaded.')

# Freeze the layers except the last 4 layers
for layer in model_vgg.layers[:-4]:
    layer.trainable = False
    
model_3 = Sequential()
model_3.add(model_vgg)

model_3.add(Flatten())
model_3.add(Dense(1024, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(6, activation='softmax'))

model_3.summary()

#Compile
model_3.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['categorical_accuracy'])

In [ ]:
checkpoint_3 = ModelCheckpoint('VGG_with_freeze.h5', save_weights_only = True, verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

model_3.fit_generator(
        train_generator,
        steps_per_epoch= train_num // batch_size,
        epochs= 25,
        validation_data=validation_generator,
        validation_steps= val_num // batch_size, 
    callbacks = [checkpoint_3])


In [ ]:
#Method 4 - Train on VGG + all layers (no freeze)

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
import numpy as np

#Get the convolutional part of a VGG network trained on ImageNet
model_vgg6_conv = VGG16(
                  include_top=False,
                  weights='imagenet',
                  input_shape= (300, 300, 3))

model_4 = Sequential()
model_4.add(model_vgg6_conv)
model_4.add(Flatten())
model_4.add(Dense(6, activation='softmax'))

model_4.summary()

#Compile
model_4.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['categorical_accuracy'])

In [ ]:
checkpoint_4 = ModelCheckpoint('VGG_all_layers.h5', save_weights_only = True, verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

model_4.fit_generator(
        train_generator,
        steps_per_epoch= train_num // batch_size,
        epochs= 30,
        validation_data=validation_generator,
        validation_steps= val_num // batch_size, 
    callbacks = [checkpoint_4])

In [ ]:
#Method 5 - Train on Inception V3 + all layers

#Build the inception v3 network
from keras.applications.inception_v3 import InceptionV3

model_inception = InceptionV3(include_top=False, weights='imagenet',input_shape=(300, 300, 3))
print('Model loaded.')

#model_inception.summary()
    
model_5 = Sequential()
model_5.add(model_inception)
model_5.add(Flatten())
model_5.add(Dense(6, activation='softmax'))

model_5.summary()

#Compile
model_5.compile(loss='categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr=1e-4),
              metrics=['categorical_accuracy'])


In [ ]:
checkpoint_5 = ModelCheckpoint('Inception_all_layers.h5', verbose=1, monitor='val_categorical_accuracy',save_best_only=True, save_weights_only = True, mode='auto')  

model_5.fit_generator(
        train_generator,
        steps_per_epoch= train_num // batch_size,
        epochs= 20,
        validation_data=validation_generator,
        validation_steps= val_num // batch_size, callbacks = [checkpoint_5])


In [ ]:
#Model 1: the performance of own model
#Accuracy on validation set = 0.422

#Model 2: the performance of Xception + full train
#Accuracy on validation set = 0.536

#Model 3: the performance of VGG 16 + freeze layers
#Accuracy on validation set = 0.551

#Model 4: the performance of Vgg16 + full train
#Accuracy on validation set = 0.476

#Model 5: the performance of InceptionV3 + full train
#Accuracy on validation set = 0.658